Technical Interview Challenge: <br>Find missing item
------

Given an list A of objects and another list B which is identical to A except that one element is removed, find that removed element.

FAQ
------

<br>
<details><summary>
What kind of objects?
</summary>
You can assume they are heterogeneous but extremely not-ill tempered. 
<br>
For right now, let's assume they either numbers or strings in the list.
<br>
Ideally, your solution would change slightly ill-tempered object types.
</details>

<br>
<details><summary>
Are the list sorted? Are the objects in the same order in each list?
</summary>
No - The objects can appear in any location.
</details>

<br>
<details><summary>
Are their duplicate items in the list?
</summary>
For right now, assume there are not duplicated items within the list. Remember all items are duplicated across lists expect for one.
</details>
<br>
<br>

Preview
------

1. Start with a working naive solution
1. Optimize that naive solution
1. Explore how the structure of the data influences possible solutions

In [226]:
reset -fs

First let's make example data

In [227]:
from random import shuffle, choice
from string import ascii_letters

In [228]:
def make_two_lists(n_items=100, hashable_items=True):
    """Make two almost identical lists, the difference is a single item is removed from one list. 
    Items in the lists are heterogeneous types."""
    
    if (n_items - len(ascii_letters)) < 0:
        # Start with letters
        a = list(ascii_letters[:n_items-1])
        # Add a single number
        a.append(1)
    else:
        # Letters + numbers to fill out
        a = list(ascii_letters) + list(range(n_items-len(ascii_letters)))    

    if not hashable_items:
        # Convert a single item to be a type that is not hashable
        a[0] = set(a[0])
        
    # Let's mix things up
    shuffle(a)
    
    # B is an independent copy of A 
    b = a[:]
    
    # Let's mix things up
    shuffle(b)
    
    # Remove one item from b
    missing_item = b.pop() 
    
    return a, b, missing_item

In [229]:
def find_missing_item_with_looping(big_list, small_list):
    "Procedurally removing common items one-by-one."
    for item in small_list:
        big_list.remove(item)
    return big_list[0]

In [231]:
# Test it for correctness
a, b, missing_item = make_two_lists()
assert find_missing_item_with_looping(big_list=a, small_list=b) == missing_item

### Analysis of time and space complexity

___Time___

O(nm<sup>*</sup>) 

n - linear time through each element in smaller list.   
m<sup>*</sup> - slightly less than linear time through m since we are decreasing its size each time.

However, over not very good because it is nested linear(ish) searches.

__Space__

O(c) - no extra space required meant. Great! 😀

In [233]:
# Benchmarking
a, b, missing_item = make_two_lists()
%timeit -n10 find_missing_item_with_looping(big_list=a, small_list=b)

ValueError: list.remove(x): x not in list

We have a ValueError when we try to benchmarking because big_list and little_list refer to (and manipulate) a and b.

We need to refactor our function for benchmarking to avoid manipulating global state.

In [234]:
def find_missing_item_with_looping_safe(big_list, small_list):
    "Procedurally removing common items one-by-one without manipulating global state."
    big_list_copy = big_list[:]
    for item in small_list:
        big_list_copy.remove(item)
    return big_list_copy[0]

In [235]:
# Benchmarking
a, b, missing_item = make_two_lists()
%timeit -n10 find_missing_item_with_looping_safe(big_list=a, small_list=b)

75.8 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Analysis of time and space complexity

___Time___

O(nm<sup>*</sup>) 

n - linear time through each element in smaller list.   
m<sup>*</sup> - slightly less than linear time through m since we are decreasing its size each time.

However, over not very good because it is nested linear(ish) searches.

__Space__

O(n) - We need extra space which will grow with the size of the list. Not great 🙁

Let's improve our run time by creating a set of copy.

We already have to pay the space cost but we can save the look up cost.

In [236]:
def find_missing_item_with_looping_optimitized(big_list, small_list):
    "Procedurally removing common items one-by-one with time optimtization."
    big_list = set(big_list)
    for item in small_list:
        big_list.remove(item)
    return big_list.pop()

In [237]:
# Test it
a, b, missing_item = make_two_lists()
assert find_missing_item_with_looping_optimitized(big_list=a, small_list=b) == missing_item

### Analysis of time and space complexity

___Time___

O(n+m) 

n - linear time through each element in smaller list.   
m - linear time to construct the set representation.

Better than before.

__Space__

O(n) - We need extra space which will grow with the size of the list. Not great 🙁

In [238]:
# Benchmarking
a, b, missing_item = make_two_lists()
%timeit -n10 find_missing_item_with_looping_optimitized(big_list=a, small_list=b)

25.1 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


The benchmarking shows we are much faster.

If that set idea works for one list representation, it will work for other...

In [239]:
def find_missing_item_with_sets(big_list, small_list):
    "Make a set comparision"
    return (set(big_list) - set(small_list)).pop()

In [240]:
# Test it
a, b, missing_item = make_two_lists()
assert find_missing_item_with_sets(big_list=a, small_list=b) == missing_item

### Analysis of time and space complexity

___Time___

Big O = O(n + m) 

n - linear time to construct set from a  
m - linear time to construct set from b  

__Space__

Big O = O(n + m)  
n - Make a hash table with entry for each time  
m - Make another hash table for each entry  

We have just about doubled our space because we have to keep to copies around

In [241]:
# Benchmarking

%timeit -n10 find_missing_item_with_sets(big_list=a, small_list=b)

9.09 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


The best time so far.

Let's compare it as items grow

In [243]:
# Benchmarking - compare the best functions as list sizes grow
n_items_list = [10, 100, 1_000, 5_000]

for n_items in n_items_list:
    print()
    print(f"n items: {n_items:,}")
    a, b, missing_item = make_two_lists(n_items)
    %timeit -n20 find_missing_item_with_looping_safe(big_list=a, small_list=b)

    a, b, missing_item = make_two_lists(n_items)
    %timeit -n20 find_missing_item_with_looping_optimitized(big_list=a, small_list=b)

    a, b, missing_item = make_two_lists(n_items)
    %timeit -n20 find_missing_item_with_sets(big_list=a, small_list=b)


n items: 10
4.06 µs ± 280 ns per loop (mean ± std. dev. of 7 runs, 20 loops each)
1.56 µs ± 75.7 ns per loop (mean ± std. dev. of 7 runs, 20 loops each)
2.25 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 20 loops each)

n items: 100
85.7 µs ± 23.9 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)
13.1 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)
8.9 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)

n items: 1,000
4.13 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)
109 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)
62.6 µs ± 10 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)

n items: 5,000
106 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)
547 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)
304 µs ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


The interesting thing is that even though the Big O optimized loop and set methods are the same, set method is just about 2x as fast.

My hypothesis is the set comparison is optimized at the byte code level compared to manually coded comparison code.

When in doubt, do the Pythonic thing to get Python's optimizations.

The current top solution assume well-tempered object types, well at least hashable.

What happens to our functions if items in the list are not hashable?

In [244]:
a, b, missing_item = make_two_lists(hashable_items=False)
find_missing_item_with_sets(big_list=a, small_list=b)

TypeError: unhashable type: 'set'

The fastest function no longer works! 🙁

In [245]:
a, b, missing_item = make_two_lists(hashable_items=False)
find_missing_item_with_looping_optimitized(big_list=a, small_list=b)

TypeError: unhashable type: 'set'

Some with our optimized loop function

In [246]:
a, b, missing_item = make_two_lists(hashable_items=False)
assert find_missing_item_with_looping_safe(big_list=a, small_list=b) == missing_item

We have to use our looping function.

It might be slow but it is robust.

Take-aways
-----

- Start with working code to understand the problem and possible solutions.
- Test, benchmark, and refactor as you go.
- Sets are useful (if you can use them).
- Knowing multiple possible solutions allows for flexibility.

References
------

- https://www.glassdoor.com/Interview/Given-an-list-A-of-objects-and-another-list-B-which-is-identical-to-A-except-that-one-element-is-removed-find-that-removed-QTN_1966980.htm

<br>
<br> 
<br>

----